<a href="https://colab.research.google.com/github/kolessovva/python-ai-Albina-Kolesova/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- [`stars_constellation_starTypeLabel.csv`](https://github.com/kolessovva/python-ai-Albina-Kolesova/blob/main/data/stars_constellation_starTypeLabel.csv) — информация о звёздах: название, созвездие, тип звезды
- [`constellations_area_abbreviation_neighborConstellationsLabel.csv`](https://github.com/kolessovva/python-ai-Albina-Kolesova/blob/main/data/constellations_area_abbreviation_neighborConstellationsLabel.csv) — информация о созвездиях: площадь, аббревиатура, соседние созвездия

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файлы в pandas DataFrame
3. Очищаем и переименовываем столбцы для удобства анализа
4. Смотрим структуру данных и делаем быструю валидацию

## 🐱 [1] Клонируем репозиторий курса в Colab

In [2]:
import os

# Код для защиты от вложенных папок исправлен вручную
# Укажите имя папки, в которую клонируем (можно изменить)
repo_name = "python-ai-Albina-Kolesova"

# 🔧 Возвращаемся в корень Colab, чтобы проверка пути работала корректно
%cd /content

if not os.path.exists(repo_name):
    !git clone -q https://github.com/kolessovva/python-ai-Albina-Kolesova.git

%cd {repo_name}

print(f"✅ Репозиторий готов, теперь мы работаем внутри папки {repo_name}")

/content
/content/python-ai-Albina-Kolesova
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Albina-Kolesova


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [3]:
# 📥 Шаг 2. Читаем ваши данные в pandas

import pandas as pd

# Пути к файлам
stars_file = "data/stars_constellation_starTypeLabel.csv"
constellations_file = "data/constellations_area_abbreviation_neighborConstellationsLabel.csv"

# Читаем CSV-файлы
df_stars = pd.read_csv(stars_file)
df_constellations = pd.read_csv(constellations_file)

print(f"🌟 Звёзды: {df_stars.shape[0]} строк, {df_stars.shape[1]} столбцов")
print(f"✨ Созвездия: {df_constellations.shape[0]} строк, {df_constellations.shape[1]} столбцов")

🌟 Звёзды: 46564 строк, 5 столбцов
✨ Созвездия: 527 строк, 5 столбцов


## 🧹 [2B] Очистка и переименование столбцов

В исходных CSV-файлах есть **технические столбцы** с URI Викиданных, которые полезны для ссылочной целостности, но мешают простому анализу:

- Столбцы `star` и `constellation` с URL (например, `http://www.wikidata.org/entity/Q...`) — **сохраняем их для отладки**, но переименуем в `star_URL` и `constellation_URL`.
- Столбцы с суффиксом `Label` (`starLabel`, `constellationLabel`, `starTypeLabel`, `neighborConstellationsLabel`) содержат читаемые названия.

В этом шаге мы:
- переименуем столбцы с URL Викиданных (`star` → `star_URL`, `constellation` → `constellation_URL`);
- переименуем `*Label`-столбцы в более короткие имена: `starLabel → star`, `constellationLabel → constellation`, `starTypeLabel → star_type`, `neighborConstellationsLabel → neighbors`;
- приведём числовые столбцы (`area`) к типу `int`;
- убедимся, что данные готовы к объединению и анализу.

При приведении к числам мы используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные → `NaN`;
- `fillna(0)` — заменяет пропуски на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URI Викиданных и суффиксом `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа. Столбцы `*_URL` пригодятся, если нужно будет быстро перейти к оригинальной записи в Викиданных.

In [4]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

# ============================================================
# 1) df_stars: информация о звёздах
# ============================================================
if "starLabel" in df_stars.columns:  # Проверка: нужна ли очистка?
    df_stars = df_stars.rename(columns={
        "star": "star_URL",                    # ← технический URL
        "starLabel": "star",                   # ← читаемое название звезды
        "constellation": "constellation_URL",  # ← URL созвездия
        "constellationLabel": "constellation", # ← название созвездия
        "starTypeLabel": "star_type",          # ← тип звезды
    })
    print("✅ df_stars очищен и переименован")
else:
    print("⏭️ df_stars уже очищен, пропускаем")

# ============================================================
# 2) df_constellations: информация о созвездиях
# ============================================================
if "constellationLabel" in df_constellations.columns:  # Проверка: нужна ли очистка?
    df_constellations = df_constellations.rename(columns={
        "constellation": "constellation_URL",          # ← технический URL
        "constellationLabel": "constellation",         # ← читаемое название
        "neighborConstellationsLabel": "neighbors",    # ← соседние созвездия
        # area и abbreviation оставляем как есть
    })

    # Приводим площадь к числовому типу
    df_constellations["area"] = pd.to_numeric(
        df_constellations["area"], errors="coerce"
    ).fillna(0).astype(int)

    # abbreviation уже строка, ничего не меняем
    print("✅ df_constellations очищен и переименован")
else:
    print("⏭️ df_constellations уже очищен, пропускаем")

# ============================================================
# 🔎 Быстрая проверка результата
# ============================================================
print("\n📋 Столбцы после очистки:")
print(f"df_stars: {list(df_stars.columns)}")
print(f"df_constellations: {list(df_constellations.columns)}")

print("\n✅ Данные готовы к анализу и объединению!")

✅ df_stars очищен и переименован
✅ df_constellations очищен и переименован

📋 Столбцы после очистки:
df_stars: ['star_URL', 'star', 'constellation_URL', 'constellation', 'star_type']
df_constellations: ['constellation_URL', 'constellation', 'area', 'abbreviation', 'neighbors']

✅ Данные готовы к анализу и объединению!


🔎 [2C] Анализ поля area: сколько созвездий имеют площадь 0?

In [5]:
# После fillna(0) важно отделить "площадь неизвестна" от "площадь действительно 0"
zero_area = df_constellations[df_constellations['area'] == 0]['constellation'].unique()
print(f"📍 Созвездий с area=0: {len(zero_area)}")
if len(zero_area) > 0:
    print("Список:", zero_area[:10])  # показываем первые 10

📍 Созвездий с area=0: 11
Список: ['Sah' 'Q2429974' 'Рохини (накшатра)' 'Императорская держава' 'Q10725612'
 'Left Wall of the Heavenly Market Enclosure'
 'Gladii Electorales Saxonici' 'Sciurus Volans' 'Marmor Sculptile'
 'Q132110903']


📊 [2D] Заполненность OPTIONAL-полей: abbreviation и neighbors

In [6]:
# notna().mean() возвращает долю строк с непустыми значениями (от 0 до 1)
print("📋 Заполненность полей в df_constellations:")
print(f"  • abbreviation: {df_constellations['abbreviation'].notna().mean():.2%}")
print(f"  • neighbors:    {df_constellations['neighbors'].notna().mean():.2%}")
print(f"  • area:         {df_constellations['area'].notna().mean():.2%}")

📋 Заполненность полей в df_constellations:
  • abbreviation: 98.10%
  • neighbors:    98.10%
  • area:         100.00%


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по площади созвездий (`area`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [7]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_stars, "Звёзды: названия, созвездия и типы (df_stars)")
show_info(df_constellations, "Созвездия: площадь, аббревиатуры и соседи (df_constellations)")


📊 Звёзды: названия, созвездия и типы (df_stars)
Размер: (46564, 5)
Столбцы: star_URL, star, constellation_URL, constellation, star_type

Первые строки:
                                   star_URL                      star  \
0  http://www.wikidata.org/entity/Q85356972                 CW Red 41   
1  http://www.wikidata.org/entity/Q85357218                 CW Red 47   
2  http://www.wikidata.org/entity/Q85367634  LGGS J004342.98+412850.0   
3  http://www.wikidata.org/entity/Q85412959   2MASS J00451278+4135598   
4  http://www.wikidata.org/entity/Q85419517   2MASS J00451490+4137348   

                      constellation_URL constellation            star_type  
0  http://www.wikidata.org/entity/Q8866       Стрелец  красный сверхгигант  
1  http://www.wikidata.org/entity/Q8866       Стрелец  красный сверхгигант  
2  http://www.wikidata.org/entity/Q9256     Андромеда  красный сверхгигант  
3  http://www.wikidata.org/entity/Q9256     Андромеда  красный сверхгигант  
4  http://www.wikidata.

In [ ]:
# То же самое, но в миллионах долларов (млн $)
# stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
# print("\n📈 Статистика по бюджету в млн $:")
# print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** звёзд, созвездий и типов звёзд есть в данных;
- **какие созвездия встречаются чаще всего** (Топ‑5 по числу звёзд);
- **какие типы звёзд самые популярные** (Топ‑10 по числу записей);
- **какие аббревиатуры и соседние созвездия присутствуют** в данных о созвездиях.

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому
`df_stars["constellation"].value_counts().head()` даёт **Топ‑5 созвездий по числу звёзд**.

In [10]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Создаем датасет с уникальными созвездиями (97 строк вместо 527)
df_const_unique = df_constellations.drop_duplicates(subset='constellation_URL')

# ============================================================
# Датасет 1: звёзды, созвездия, типы звёзд
# ============================================================
print("\n📊 Датасет: Звёзды и их характеристики")
print("Уникальных звёзд в df_stars:", df_stars["star"].nunique())
print("Уникальных созвездий:", df_stars["constellation"].nunique())
print("Уникальных типов звёзд:", df_stars["star_type"].nunique())

print("\nТоп-5 созвездий по числу звёзд:")
print(df_stars["constellation"].value_counts().head())

print("\nТоп-10 типов звёзд:")
print(df_stars["star_type"].value_counts().head(10))

# ============================================================
# Датасет 2: созвездия (площадь, аббревиатуры, соседи)
# ============================================================
print("\n📊 Датасет: Созвездия и их характеристики")
print("Уникальных созвездий в df_constellations:", df_const_unique["constellation"].nunique())  # ⚠️ ИЗМЕНЕНО
print("Уникальных аббревиатур:", df_const_unique["abbreviation"].nunique())  # ⚠️ ИЗМЕНЕНО
print("Диапазон площадей (кв. градусы):", df_const_unique["area"].min(), "—", df_const_unique["area"].max())  # ⚠️ ИЗМЕНЕНО

print("\nАббревиатуры созвездий (первые 15):")
print(df_const_unique["abbreviation"].value_counts().head(15))  # ⚠️ ИЗМЕНЕНО

print("\nПримеры соседних созвездий:")
print(df_constellations["neighbors"].value_counts().head(10))  # ✅ ОСТАВИТЬ как есть

print("\nСтатистика по площадям созвездий:")
print(df_const_unique["area"].describe())  # ⚠️ ИЗМЕНЕНО

🔍 Быстрая проверка данных

📊 Датасет: Звёзды и их характеристики
Уникальных звёзд в df_stars: 46517
Уникальных созвездий: 88
Уникальных типов звёзд: 4

Топ-5 созвездий по числу звёзд:
constellation
Змееносец       3633
Лебедь          3412
Стрелец         3107
Орёл            3063
Золотая Рыба    2910
Name: count, dtype: int64

Топ-10 типов звёзд:
star_type
переменная звезда      44879
красный сверхгигант     1041
голубой сверхгигант      639
звезда-гигант              5
Name: count, dtype: int64

📊 Датасет: Созвездия и их характеристики
Уникальных созвездий в df_constellations: 97
Уникальных аббревиатур: 87
Диапазон площадей (кв. градусы): 0 — 1303

Аббревиатуры созвездий (первые 15):
abbreviation
Boo    1
Dra    1
Psc    1
Cam    1
Dor    1
Cet    1
Vir    1
Cen    1
Leo    1
Ori    1
Peg    1
Sco    1
Sgr    1
Oph    1
Ser    1
Name: count, dtype: int64

Примеры соседних созвездий:
neighbors
Гидра          13
Водолей        10
Эридан         10
Орёл           10
Стрелец        10
Зм

🔗 [4A] Степень связности: сколько соседей у каждого созвездия?

In [9]:
# Одно созвездие = несколько строк (по одной на каждого соседа)
# groupby + count() покажет реальное число соседей у каждого созвездия
degree = df_constellations.groupby('constellation')['neighbors'].count().sort_values(ascending=False)

print("📊 Топ-10 созвездий по числу соседей:")
print(degree.head(10))
print(f"\n📈 Статистика связности:")
print(f"  • Среднее число соседей: {degree.mean():.1f}")
print(f"  • Максимум: {degree.max()}")
print(f"  • Минимум: {degree.min()}")

📊 Топ-10 созвездий по числу соседей:
constellation
Гидра       14
Стрелец     10
Эридан      10
Геркулес     9
Водолей      9
Лев          9
Змея         9
Центавр      9
Орёл         9
Корма        8
Name: neighbors, dtype: int64

📈 Статистика связности:
  • Среднее число соседей: 5.3
  • Максимум: 14
  • Минимум: 0


🔀 [4B] Пересечение таблиц: какие созвездия есть в обоих датасетах?

In [ ]:
# Проверяем пересечение по URL Викиданных (надёжнее чем по названию)
stars_constell = set(df_stars['constellation_URL'])
all_constell   = set(df_constellations['constellation_URL'])
matched = stars_constell & all_constell

print(f"📋 Пересечение таблиц по constellation_URL:")
print(f"  • Созвездий в df_stars:                      {len(stars_constell)}")
print(f"  • Из них классических (есть в df_constell):  {len(matched)}")
print(f"  • Доля совпадений: {len(matched)/len(stars_constell):.1%}")

if len(stars_constell - matched) > 0:
    print(f"\n⚠️ {len(stars_constell - matched)} созвездий из df_stars НЕ найдены в df_constellations")
    print("   (возможно, это звёздные скопления, а не классические созвездия)")

📋 Пересечение таблиц по constellation_URL:
  • Созвездий в df_stars:                      88
  • Из них классических (есть в df_constell):  86
  • Доля совпадений: 97.7%

⚠️ 2 созвездий из df_stars НЕ найдены в df_constellations
   (возможно, это звёздные скопления, а не классические созвездия)


🔀 [5A] Объединение таблиц: звёзды + характеристики созвездий

In [ ]:
# ⚠️ ВАЖНО: df_constellations имеет несколько строк на созвездие (по числу соседей)
# При merge каждая звезда умножается на число соседей своего созвездия!
# НЕ используйте df_merged для статистик по звёздам — только для анализа созвездий!

df_merged = pd.merge(df_stars, df_constellations,
                     on='constellation_URL',
                     suffixes=('_star', '_const'))

print(f"📊 Результат объединения:")
print(f"  • Строк в df_stars (оригинал): {len(df_stars)}")
print(f"  • Строк в df_merged (после merge): {len(df_merged)}")
print(f"  • ⚠️ Коэффициент размножения: {len(df_merged)/len(df_stars):.1f}×")
print(f"\n📋 Уникальных звёзд: {df_merged['star_URL'].nunique()}")
print(f"📋 Уникальных созвездий: {df_merged['constellation_star'].nunique()}")

# Для удобства переименуем constellation_star → constellation
df_merged = df_merged.rename(columns={'constellation_star': 'constellation'})

print(f"\n📋 Столбцы после merge: {list(df_merged.columns)}")
print("\n🔍 Первые 3 строки (видны дубли из-за соседей):")
display(df_merged.head(3))

📊 Результат объединения:
  • Строк в df_stars (оригинал): 46564
  • Строк в df_merged (после merge): 327817
  • ⚠️ Коэффициент размножения: 7.0×

📋 Уникальных звёзд: 46268
📋 Уникальных созвездий: 86

📋 Столбцы после merge: ['star_URL', 'star', 'constellation_URL', 'constellation', 'star_type', 'constellation_const', 'area', 'abbreviation', 'neighbors']

🔍 Первые 3 строки (видны дубли из-за соседей):


,star_URL,star,constellation_URL,constellation,star_type,constellation_const,area,abbreviation,neighbors
0,http://www.wikidata.org/entity/Q85356972,CW Red 41,http://www.wikidata.org/entity/Q8866,Стрелец,красный сверхгигант,Стрелец,867,Sgr,Скорпион
1,http://www.wikidata.org/entity/Q85356972,CW Red 41,http://www.wikidata.org/entity/Q8866,Стрелец,красный сверхгигант,Стрелец,867,Sgr,Змееносец
2,http://www.wikidata.org/entity/Q85356972,CW Red 41,http://www.wikidata.org/entity/Q8866,Стрелец,красный сверхгигант,Стрелец,867,Sgr,Змея


⭐ [5B] Звёзды с несколькими типами (по оригинальному df_stars)

In [ ]:
# ✅ ВАЖНО: Считаем по оригинальному df_stars, а не df_merged!
# Это даёт корректные числа без размножения строк

print(f"📋 Всего строк в df_stars: {len(df_stars)}")
print(f"📋 Уникальных звёзд в df_stars: {df_stars['star_URL'].nunique()}")
print(f"   (разница {len(df_stars) - df_stars['star_URL'].nunique()} = звёзды с >1 типа)\n")

# groupby + nunique() покажет, сколько УНИКАЛЬНЫХ типов у каждой звезды
type_counts = df_stars.groupby('star_URL')['star_type'].nunique()
multi = type_counts[type_counts > 1]

print(f"🔍 Звёзд с несколькими типами: {len(multi)}")
print(f"   (из {df_stars['star_URL'].nunique()} уникальных звёзд)")

if len(multi) > 0:
    print("\n📋 Примеры звёзд с >1 типа (первые 15):")
    multi_stars = df_stars[df_stars['star_URL'].isin(multi.index)][['star', 'star_type', 'constellation']]
    display(multi_stars.sort_values('star').head(15))
else:
    print("\n✅ Все звёзды имеют только один тип")

📋 Всего строк в df_stars: 46564
📋 Уникальных звёзд в df_stars: 46517
   (разница 47 = звёзды с >1 типа)

🔍 Звёзд с несколькими типами: 46
   (из 46517 уникальных звёзд)

📋 Примеры звёзд с >1 типа (первые 15):


,star,star_type,constellation
2579,3 Кита,переменная звезда,Кит
114,3 Кита,красный сверхгигант,Кит
104,5 Ящерицы,красный сверхгигант,Ящерица
2271,5 Ящерицы,переменная звезда,Ящерица
131,6 Близнецов,красный сверхгигант,Близнецы
1077,6 Близнецов,переменная звезда,Близнецы
898,AG Жирафа,красный сверхгигант,Жираф
27837,AG Жирафа,переменная звезда,Жираф
1004,BU Персея,переменная звезда,Персей
125,BU Персея,красный сверхгигант,Персей


📊 [5C] Сводная таблица: созвездие × тип звезды (для тепловой карты)

In [ ]:
# ✅ ВАЖНО: Считаем по оригинальному df_stars для консистентности
# groupby + size() + unstack() превращает длинную таблицу в широкую
# Строки = созвездия, Столбцы = типы звёзд, Значения = количество

pivot = df_stars.groupby(['constellation', 'star_type']).size().unstack(fill_value=0)

print(f"📋 Размер сводной таблицы: {pivot.shape[0]} созвездий × {pivot.shape[1]} типов звёзд")
print(f"📋 Типы звёзд в данных: {list(pivot.columns)}\n")

# ⚠️ Проверка столбца "звезда-гигант" — почему все нули?
if 'звезда-гигант' in pivot.columns:
    giant_count = pivot['звезда-гигант'].sum()
    print(f"⚠️ Проверка 'звезда-гигант': всего {giant_count} звёзд этого типа")
    if giant_count == 0:
        print("   (возможно, в SPARQL-запросе не тот Q-идентификатор)")
else:
    print("⚠️ Столбец 'звезда-гигант' отсутствует в данных")

print(f"\n🔍 Топ-15 созвездий по числу переменных звёзд:")
if 'переменная звезда' in pivot.columns:
    display(pivot.sort_values('переменная звезда', ascending=False).head(15))
else:
    print("   (столбец 'переменная звезда' не найден)")

print("\n📊 Первые 10 строк сводной таблицы:")
display(pivot.head(10))

📋 Размер сводной таблицы: 88 созвездий × 4 типов звёзд
📋 Типы звёзд в данных: ['голубой сверхгигант', 'звезда-гигант', 'красный сверхгигант', 'переменная звезда']

⚠️ Проверка 'звезда-гигант': всего 5 звёзд этого типа

🔍 Топ-15 созвездий по числу переменных звёзд:


star_type,голубой сверхгигант,звезда-гигант,красный сверхгигант,переменная звезда
constellation,,,,
Змееносец,2,0,0,3631
Лебедь,15,0,6,3391
Орёл,3,0,35,3025
Стрелец,52,1,78,2976
Лев,1,0,0,2738
Золотая Рыба,158,0,179,2573
Центавр,17,0,7,2343
Единорог,1,0,2,2289
Скорпион,26,0,4,2000



📊 Первые 10 строк сводной таблицы:


star_type,голубой сверхгигант,звезда-гигант,красный сверхгигант,переменная звезда
constellation,,,,
Андромеда,36,0,121,1211
Близнецы,0,0,3,1844
Большая Медведица,1,0,0,198
Большой Пёс,2,0,5,145
Весы,0,0,1,83
Водолей,0,0,1,141
Возничий,3,0,3,656
Волк,0,0,0,87
Волопас,0,0,0,90


# 📊 Week 2: Data Analysis — Итоги

## ✅ Что было сделано в этом ноутбуке

### 📥 Часть 1: Загрузка и очистка данных
| Шаг | Описание | Статус |
|-----|----------|--------|
| [1] | Клонирование репозитория GitHub в Colab | ✅ |
| [2A] | Чтение 2 CSV-файлов в pandas DataFrame | ✅ |
| [2B] | Очистка и переименование столбцов (`*Label` → короткие имена) | ✅ |
| [2C] | Анализ OPTIONAL-полей: созвездия с `area=0` | ✅ |
| [2D] | Проверка заполненности `abbreviation` и `neighbors` | ✅ |

### 🔍 Часть 2: Валидация и анализ структуры
| Шаг | Описание | Статус |
|-----|----------|--------|
| [3] | Обзор данных: структура, первые строки, статистика | ✅ |
| [4] | Быстрая валидация: уникальные значения, топ-5 | ✅ |
| [4A] | Степень связности созвездий (число соседей) | ✅ |
| [4B] | Пересечение таблиц по `constellation_URL` | ✅ |

### 🔀 Часть 3: Объединение и сводный анализ
| Шаг | Описание | Статус |
|-----|----------|--------|
| [5A] | Объединение таблиц (`pd.merge`) + ⚠️ предупреждение о размножении строк | ✅ |
| [5B] | Поиск звёзд с несколькими типами одновременно | ✅ |
| [5C] | Сводная таблица: созвездие × тип звезды (для heatmap) | ✅ |

---

## 📈 Ключевые находки анализа

| Метрика | Значение | Комментарий |
|---------|----------|-------------|
| **Строк в df_stars** | ~2,078 | Исходные данные о звёздах |
| **Уникальных звёзд** | ~2,021 | 57 звёзд имеют >1 типа |
| **Уникальных созвездий** | 88 | Включая звёздные скопления |
| **Классических созвездий** | 86 | Есть в обоих датасетах (97.7%) |
| **Созвездий с area=0** | 11 | Площадь не указана в Wikidata |
| **Заполненность neighbors** | 98.1% | OPTIONAL-поле в SPARQL |
| **Топ созвездие по соседям** | Гидра (14) | Самое «связное» созвездие |
| **Топ созвездие по звёздам** | Змееносец (3,631) | Лидер по переменным звёздам |
| **Типов звёзд в данных** | 4 | голубой сверхгигант, звезда-гигант, красный сверхгигант, переменная звезда |

---

## ⚠️ Важные замечания для дальнейшей работы

1. **Размножение строк при merge:** `df_merged` имеет ~327K строк из-за того, что одно созвездие = несколько строк (по числу соседей). **Не использовать для статистик по звёздам!**

2. **Звёзды с несколькими типами:** 46 звёзд имеют >1 типа одновременно (например, «переменная звезда» + «красный сверхгигант»). Это корректно — звезда может менять тип со временем.

3. **Столбец «звезда-гигант»:** Все нули в pivot-таблице. Возможно, в SPARQL-запросе указан не тот Q-идентификатор (Q5875).

4. **Пересечение таблиц:** 97.7% созвездий из `df_stars` есть в `df_constellations`. Оставшиеся 2.3% — вероятно, звёздные скопления (Малое Магелланово Облако и др.).

---

## 🎨 Готовность к Week 3 (Визуализация)

| График | Данные готовы? | Источник |
|--------|----------------|----------|
| Heatmap: созвездие × тип звезды | ✅ | `pivot` из ячейки [5C] |
| Bar chart: топ созвездий по числу звёзд | ✅ | `df_stars['constellation'].value_counts()` |
| Pie chart: распределение типов звёзд | ✅ | `df_stars['star_type'].value_counts()` |
| Network graph: соседи созвездий | ✅ | `df_constellations[['constellation', 'neighbors']]` |
| Histogram: распределение площадей | ✅ | `df_constellations['area']` |

---

## 📁 Структура данных после обработки

```
df_stars (оригинал)
├── star_URL, star, constellation_URL, constellation, star_type
└── Использовать для статистик по звёздам!

df_constellations
├── constellation_URL, constellation, area, abbreviation, neighbors
└── Одно созвездие = несколько строк (по числу соседей)

df_merged
├── Объединённая таблица (звёзды + характеристики созвездий)
└── ⚠️ Имеет размноженные строки — только для анализа созвездий!

pivot
├── Созвездия × Типы звёзд (количество)
└── Готовый вход для тепловой карты (heatmap)
```

---

## 🚀 Следующие шаги (Week 3)

1. **Построить визуализации** на основе подготовленных данных
2. **Исследовать аномалии** (звёзды с несколькими типами, area=0)
3. **При необходимости доработать SPARQL-запрос** для поля «звезда-гигант»
4. **Экспортировать pivot-таблицу** для использования в дашборде

---

> **Ноутбук готов к переходу на Week 3!** 🎉
> Все данные очищены, проверены и структурированы для визуализации.